Output of this script can be loaded from pickle file (predicted_TFBS_normalized.pkl) by executing the last cell.

In [1]:
import math
import pandas as pd
from TF import *

In [2]:
finalResults = pd.DataFrame()

for i in range(10):
    print('Reading file ', i+1, 'of 10')
    data = pd.read_csv('deepsea-results/'+str(i)+'-infile.bed.out', header=0, index_col=0)
    results = data.iloc[:,0:3]
    for TF in TFsSlash:
        results = results.join(data.filter(regex=TF).iloc[:,0])
    if i == 0:
        finalResults = results
    else:
        finalResults = finalResults.append(results, ignore_index=True)
finalResults.head(-1)

Reading file  1 of 10
Reading file  2 of 10
Reading file  3 of 10
Reading file  4 of 10
Reading file  5 of 10
Reading file  6 of 10
Reading file  7 of 10
Reading file  8 of 10
Reading file  9 of 10
Reading file  10 of 10


,chr,start,end,GM12878|BRCA1|None,GM12801|CTCF|None,K562|E2F4|None,GM12878|c-Fos|None,HeLa-S3|IRF3|None,H1-hESC|SP4|None,GM12891|TAF1|None,K562|STAT2|IFNa6h,T-47D|ERalpha|Genistein_100nM,HEK293|KAP1|None
0,chr8,2,1001,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
1,chr8,202,1201,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
2,chr8,402,1401,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
3,chr8,602,1601,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
4,chr8,802,1801,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
...,...,...,...,...,...,...,...,...,...,...,...,...,...
731804,chr8,146360802,146361801,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
731805,chr8,146361002,146362001,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
731806,chr8,146361202,146362201,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546
731807,chr8,146361402,146362401,0.000328,0.00005,0.000249,0.000371,0.000218,0.0004,0.001367,0.000362,0.001316,0.008546


Normalize the probabilities to take into account each TF's positive rate.

In [3]:
# positive proportion given by experimental data
posRate = pd.read_csv('positiveProportion.txt', header=0, delimiter='\t')
TFposRate = []
TFposRate.append(posRate[(posRate['Cell Type'] == 'GM12878') & (posRate['TF/DNase/HistoneMark'] == 'BRCA1') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'GM12801') & (posRate['TF/DNase/HistoneMark'] == 'CTCF') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'HepG2') & (posRate['TF/DNase/HistoneMark'] == 'Nrf1') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'K562') & (posRate['TF/DNase/HistoneMark'] == 'E2F4') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'GM12878') & (posRate['TF/DNase/HistoneMark'] == 'c-Fos') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'HeLa-S3') & (posRate['TF/DNase/HistoneMark'] == 'IRF3') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'H1-hESC') & (posRate['TF/DNase/HistoneMark'] == 'SP4') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'GM12891') & (posRate['TF/DNase/HistoneMark'] == 'TAF1') & (posRate['Treatment'] == 'None')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'K562') & (posRate['TF/DNase/HistoneMark'] == 'STAT2') & (posRate['Treatment'] == 'IFNa6h')]['Positive Proportion'].item())
TFposRate.append(posRate[(posRate['Cell Type'] == 'K562') & (posRate['TF/DNase/HistoneMark'] == 'ZNF274') & (posRate['Treatment'] == 'None')]['Positive Proportion'].tolist()[0])
TFposRate = pd.DataFrame(TFposRate, columns=['Positive Rate'], index=TFs)
TFposRate.head(10)

,Positive Rate
GM12878|BRCA1|None,0.000309
GM12801|CTCF|None,0.001411
K562|E2F4|None,0.001026
GM12878|c-Fos|None,0.004917
HeLa-S3|IRF3|None,0.001092
H1-hESC|SP4|None,0.000986
GM12891|TAF1|None,0.006530
K562|STAT2|IFNa6h,0.007451
T-47D|ERalpha|Genistein_100nM,0.001165
HEK293|KAP1|None,0.002650


In [4]:
# normalizing as a function of positive proportion in experimental data
for i, index in enumerate(TFposRate.index):
    print('Normalizing data for TF ', index, ' (', i+1,'/10)')
    myitem = TFposRate.iloc[i].item()
    # finalResults[index] = finalResults[index].map(lambda P:1/(1+math.exp(-(math.log(P/(1-P))+math.log(0.05/(1-0.05))-math.log(TFposRate.iloc[i].item()/(1-TFposRate.iloc[i].item()))))))
    finalResults[index] = finalResults[index].map(lambda P:1/(1+math.exp(-(math.log(P/(1-P))+math.log(0.05/(1-0.05))-math.log(myitem/(1-myitem))))))
finalResults.head(-1)

Normalizing data for TF  GM12878|BRCA1|None  ( 1 /10)
Normalizing data for TF  GM12801|CTCF|None  ( 2 /10)
Normalizing data for TF  K562|E2F4|None  ( 3 /10)
Normalizing data for TF  GM12878|c-Fos|None  ( 4 /10)
Normalizing data for TF  HeLa-S3|IRF3|None  ( 5 /10)
Normalizing data for TF  H1-hESC|SP4|None  ( 6 /10)
Normalizing data for TF  GM12891|TAF1|None  ( 7 /10)
Normalizing data for TF  K562|STAT2|IFNa6h  ( 8 /10)
Normalizing data for TF  T-47D|ERalpha|Genistein_100nM  ( 9 /10)
Normalizing data for TF  HEK293|KAP1|None  ( 10 /10)


,chr,start,end,GM12878|BRCA1|None,GM12801|CTCF|None,K562|E2F4|None,GM12878|c-Fos|None,HeLa-S3|IRF3|None,H1-hESC|SP4|None,GM12891|TAF1|None,K562|STAT2|IFNa6h,T-47D|ERalpha|Genistein_100nM,HEK293|KAP1|None
0,chr8,2,1001,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
1,chr8,202,1201,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
2,chr8,402,1401,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
3,chr8,602,1601,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
4,chr8,802,1801,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
...,...,...,...,...,...,...,...,...,...,...,...,...,...
731804,chr8,146360802,146361801,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
731805,chr8,146361002,146362001,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
731806,chr8,146361202,146362201,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839
731807,chr8,146361402,146362401,0.052854,0.001877,0.012621,0.003937,0.010391,0.020903,0.010841,0.00253,0.056122,0.145839


In [5]:
# save the resulting table to a pickle file
finalResults.to_pickle('predicted_TFBS_normalized_0-9_final10_low.pkl')

In [6]:
# load results from pickle file
#finalResults = pd.read_pickle('predicted_TFBS_normalized_0-9.pkl')
#finalResults.head(-1)